In [147]:
import requests 
import time
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
from pyshadow.main import Shadow


In [148]:
driver = webdriver.Chrome()
shadow = Shadow(driver)

In [149]:
def get_all_products(search_item:str):
    is_there_elements = True
    page_index = 1

    all_products = []

    while is_there_elements:
        url = f'https://www.maxtitanium.com.br/busca?q={search_item.replace(" ", "+")}&page={page_index}'
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        time.sleep(3)

        elements = shadow.find_elements('div.content')

        is_there_elements = len(elements) > 0

        if is_there_elements:
            for i, el in enumerate(elements):
                unpacked_items = el.text.split('\n')

                if len(unpacked_items) == 2:
                    product_name = unpacked_items[0]
                    current_price = None
                    last_price = None
                    availability = False
                else:
                    product_name, current_price, last_price = unpacked_items
                    availability = True

                try:
                    last_price = float(last_price.replace('R$', '').replace(',', '.'))
                    current_price = float(current_price.replace('R$', '').replace(',', '.'))
                except:
                    last_price = np.NaN
                    current_price = np.NaN
                
                
                product_dict = {
                    'Nome_produto': product_name,
                    'Preco_anterior': last_price,
                    'Preco_atual': current_price,
                    'Produto_disponivel': availability
                }


                all_products.append(product_dict)
        
        page_index += 1
    
    return pd.DataFrame(all_products)

In [150]:
# items = [
#     {'descrp': 'item 1', 'vl': 'R$100,00'},
#     {'descrp': 'item 1', 'vl': 'R$120,00'},
#     {'descrp': 'item 1', 'vl': 'R$130,00'},
#     {'descrp': 'item 1', 'vl': None}
# ]

In [151]:
# for i in items:
#     try:
#         i['vl'] = float(i['vl'].replace('R$', '').replace(',', '.'))
#     except:
#         i['vl'] = np.nan
    
#     print(i['vl'])

In [152]:
hipercaloricos = get_all_products('hipercalorico')

In [ ]:
print('qnt items:', hipercaloricos.shape[0])

hipercaloricos

In [158]:
url = 'https://www.maxtitanium.com.br/masstitanium3kg/p'
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
print(type(soup))
elements = soup.find_all(class_='vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--specification pb0')

print(elements[0].get_text())

# for el in elements:
#     print(el.text)

<class 'bs4.BeautifulSoup'>

